# Tutorial 05, Part 1: Pandas DataFrames 
[The official project homepage](https://pandas.pydata.org)

* Goal
    * Extend what we learned about Series objects in the previous tutorial to their 2D counterpart - DataFrames
    * Develop some tools for dealing with missing data (not exhaustive, but a good start)
    * Take this chance to also learn a bit about file input/output (I/O) and some other more advanced coding techniques

## DataFrames

[Pandas quick start guide for DataFrames](https://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe)

* A DataFrame (DF) is a labeled data struture that can be thought of as a 2D extension of the Series objects that we discussed in the first part of the tutorial
* A DF can accept many types of input, from a 2D ndarray, multiple Series, a dict of 1D arrays, another DF, etc
* Like a Series, DFs contain data values and their labels. Because we're now dealing with a 2D structure, we call the **row labels the index argument** and the **column labels the column argument**. 
    * Like a Series, if you don't explicitly assign row and column labels, then they will be auto-generated (but not as useful as specifying the labels yourself!)

<div class="alert alert-info">
Much of what we learned about Series objects will generalize to DFs, so here we'll focus on some of key functionality that might not be obvious based on the previous tutorial.
</div>

<div class="alert alert-info">
One more quick note: if using an older version of Python (earlier than 3.6) and Pandas (earlier than 0.23) and you create a DF from a dict without explicitly specifying column names, then the column names will be entered into the DF based on lexical order
</div>

## Import libs

In [1]:
# standard numpy module
import numpy as np

# import a generic pandas object and also a few specific functions that we'll use
import pandas as pd 
from pandas import DataFrame, read_csv

# new - get and store current file path for file i/o later on in tutorial
import os
cwd = os.getcwd()

## Make up a data set to demonstrate functionality, will import some real data later on
* Here we'll pretend that we did a unit recording experiment 
    * There are two stimulus conditions
    * And we are recording from 10 different neurons 

In [2]:
# seed random number generator so that we're all seeing the same thing in class
np.random.RandomState(0)

In [4]:
# index lables for our 10 neurons...see previous tutorial for more elegant ways of generating
# index labels, here we're just going to write them out
neuron_labels = ['Nrn0', 'Nrn1','Nrn2','Nrn3','Nrn4','Nrn5','Nrn6','Nrn7','Nrn8','Nrn9']  

In [5]:
# generate response in each neuron to stimulus 1...
resp1_hz = [14, 27, 62, 88, 45, 56, 75, 63, 33, 46]

In [6]:
# generate a response to stimulus 2...use random.randint just for practice/fun
min_resp = 0  # inclusive
max_resp = 90 # exclusive
resp2_hz = np.random.randint(min_resp, max_resp, len(resp1_hz))

## New - use 'zip' function to wrap up the data from each list into one list
[reference page for zip](https://www.w3schools.com/python/ref_func_zip.asp)

* Operates just like it sounds  - takes a set of iterators and groups them together into a single iterator with the 1st element in the resultant iterator comprised of the first element of each iterator 'zipped' together, then the second element from each iterator zipped together, etc. 
* Length of resulting iterator limited by the length of the shortest input iterator!

<div class="alert alert-warning">
Because the length of the resulting iterator is limited by length of shortest input iterator, you can sometimes not get an error if you try to zip together iterators with unequal lengths - this is fine if intenitonal, but if the unequal length was caused by a bug, then you may not find it when using zip!
</div>

In [7]:
neuron_data = list(zip(resp1_hz, resp2_hz))
print(neuron_data)

print('Grab one index to see the two response arrays zipped together:')
print(neuron_data[9])

[(14, 31), (27, 40), (62, 14), (88, 51), (45, 21), (56, 82), (75, 88), (63, 50), (33, 61), (46, 39)]
Grab one index to see the two response arrays zipped together:
(46, 39)


In [16]:
# note: to unzip go like this and you'll get back the original
uz_data1, uz_data2 = zip(*neuron_data)

print(uz_data1,"\n\n",uz_data2)

(14, 27, 62, 88, 45, 56, 75, 63, 33, 46) 

 (31, 40, 14, 51, 21, 82, 88, 50, 61, 39)


## Make a DataFrame object to hold the contents of the data set
[DataFrame help page](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.html)

* Just like with the pd.Series call, you specify the data, index labels (row labels in this case)
* In addition to row labels, you can also specify column labels (with 'columns')
* Can also specify data type (default is inferred)
* Like pd.Series you can ask for an independent copy of the data (copy=True) or you will get a view by default (i.e. copy=False)

In [19]:
# make the call to pd.DataFrames to create the DF - usage much like pd.Series
df = pd.DataFrame(data = neuron_data, index=neuron_labels, columns = ['resp1', 'resp2'])

# take a look at the output...
display(df)   # compare to print(df) - looks nicer with display thanks to iPython backend 

,resp1,resp2
Nrn0,14,31
Nrn1,27,40
Nrn2,62,14
Nrn3,88,51
Nrn4,45,21
Nrn5,56,82
Nrn6,75,88
Nrn7,63,50
Nrn8,33,61
Nrn9,46,39


In [20]:
# another handy display function...good for large dfs that are too big to fit - at least you can get an idea of 
# the overall structure
df.head()

,resp1,resp2
Nrn0,14,31
Nrn1,27,40
Nrn2,62,14
Nrn3,88,51
Nrn4,45,21


## Get a high-level summary of the data using built-in functionality of DataFrame object
[API reference page](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html)

In [21]:
# first call this using the defaults
df.describe()

,resp1,resp2
count,10.000000,10.000000
mean,50.900000,47.700000
std,22.521348,24.166322
min,14.000000,14.000000
25%,36.000000,33.000000
50%,51.000000,45.000000
75%,62.750000,58.500000
max,88.000000,88.000000


In [22]:
# can specify different values to change behavior...
df.describe(percentiles=np.linspace(0,1,11))

,resp1,resp2
count,10.000000,10.000000
mean,50.900000,47.700000
std,22.521348,24.166322
min,14.000000,14.000000
0%,14.000000,14.000000
10%,25.700000,20.300000
20%,31.800000,29.000000
30.0%,41.400000,36.600000
40%,45.600000,39.600000
50%,51.000000,45.000000


<div class="alert alert-warning">

BEGIN SIDEBAR - Common source of NaNs in data and how to deal with them in general

Important bit of info for avoiding a common source of confusion (and potential bugs!!!)


* Note that if you make a DF out of a set of Series (e.g. a dict of Series), then the resulting DF index labels will be the union of the index labels in each Series
* This can be confusing because the DF will still be formed even if you have mismatching labels or even if you have two series of different sizes..
* Fortunately, the misaligned (or missing) values will be filled in with NaNs ('Not-a-Number') to serve as a placeholder for the misaligned or missing info
* Quick demo below before continuing on with our sample neuron data from above 
</div>

In [24]:
# make a set of two Series with unequal lengths stored in a dict, 
# with each Series having data and index labels

# Note that Series 1 has 4 elements, but Series 2 has 5 elements!

data_dict = {'dict0' : pd.Series(data = np.random.randn(4), index=['0','1','2','3']), 
            'dict1' : pd.Series(data = np.random.randint(0,5,5), index=['0','1','2','3','4'])}

# make a data frame
weird_df = pd.DataFrame(data_dict)

# take a look - notice that pd.DataFrame did not throw an error even though
# the input Series are different sizes...however, it did mark the missing value 
# with a NaN
display(weird_df)

,dict0,dict1
0,-1.254777,1
1,-0.739414,3
2,1.200231,0
3,-0.668611,1
4,NaN,0


<div class = "alert alert-info">
Because of the above behavior, often good to frequently check for NaNs in your data to identify processing steps that might have gone awry...(unless you are expecting NaNs as part of routine processing, in which case this might not be very helpful). 
Many ways to do this, but here is one way that works pretty well with identifying the presence of a NaN anywhere in the DataFrame (which is handy beacuse often times the DataFrames are too large to easily see in their entirety) 
</div>

In [25]:
# show True/False for each element in DF
# first do it the NumPy way
display(np.isnan(weird_df))

# use the 'any' method to figure out if any entries are NaN
if np.isnan(weird_df).any:
    print('weird...you have NaNs in your data')

,dict0,dict1
0,False,False
1,False,False
2,False,False
3,False,False
4,True,False


weird...you have NaNs in your data


### Pandas also provides functions for dealing with NaNs (or missing values)

In [26]:
# return true for NaNs
pd.isna(weird_df)

,dict0,dict1
0,False,False
1,False,False
2,False,False
3,False,False
4,True,False


In [27]:
# Or return true for non NaNs
pd.notna(weird_df)  

,dict0,dict1
0,True,True
1,True,True
2,True,True
3,True,True
4,False,True


In [28]:
# apply to just one column at a time
# note that you can call the function from the object
# directly (instead of using the pd object)
weird_df['dict0'].notna()

0     True
1     True
2     True
3     True
4    False
Name: dict0, dtype: bool

### Interpolate across missing values...
* Default is linear, but can do all sorts of other interpolations using scipy

In [29]:
norm_df = weird_df.interpolate()
display(weird_df)
display(norm_df)
# note that since the missing value is the last in the series
# it just takes the value of the 2nd to last entry
# otherwise it would be linearly interpolated by default

,dict0,dict1
0,-1.254777,1
1,-0.739414,3
2,1.200231,0
3,-0.668611,1
4,NaN,0


,dict0,dict1
0,-1.254777,1
1,-0.739414,3
2,1.200231,0
3,-0.668611,1
4,-0.668611,0


### Can also fill NaNs lots of other ways...like assigning the mean of all or some of the data to NaNs

In [30]:
# e.g. fill the NaN with the mean of the first column!
weird_df.fillna(weird_df.mean()['dict0'])

,dict0,dict1
0,-1.254777,1
1,-0.739414,3
2,1.200231,0
3,-0.668611,1
4,-0.365643,0


<div class = "alert alert-info">

This is just a small subset of methods for finding and dealing with missing data. See this link for a more complete set of possibilities:

[Pandas missing data](https://pandas.pydata.org/pandas-docs/stable/missing_data.html)

Be careful rooting out NaNs using equivalence testing - NaNs don't compare equal (cause they are missing or unknown values). Better to use boolean indexing based on the return vector from np.isnan or pd.isna, etc


Remember that in most cases:
* When summing data, NaNs will be treated as zero.
* Functions like cumsum() and cumprod() ignore NaNs by default, but will propogate them to returned arrays. 
    * To change behaviour to include NaNs, often use skipna=False or similar.
* If multiplying by NaN, the result is generally NaN
    * Except if computing the prod of all NaNs, in which case the answer is 1!
</div>

In [31]:
# make an array of nans
x = np.repeat(np.nan, 10)
print(x[0]*x[1])  # NaN

# convert x to a series
s = pd.Series(x)
# compute the prod of elements in series
pd.Series.prod(s)

nan


1.0

<div class="alert alert-warning">
END SIDEBAR...now back to the main thread and our 10 neuron experimental data
</div>

## Indexing, adding, deleting entire columns
* Think of the DF as a dict of Series objects with common labels - much of the syntax is the same as for dicts (and for Series)

In [32]:
# grab the second column from our DF
display(df['resp2'])

Nrn0    31
Nrn1    40
Nrn2    14
Nrn3    51
Nrn4    21
Nrn5    82
Nrn6    88
Nrn7    50
Nrn8    61
Nrn9    39
Name: resp2, dtype: int64

### Adding a column is easy and can be done dynamically (on the fly)

In [33]:
# define a third response column as the product of the first two columns
df['resp3'] = df.resp1 * df.resp2
display(df)

,resp1,resp2,resp3
Nrn0,14,31,434
Nrn1,27,40,1080
Nrn2,62,14,868
Nrn3,88,51,4488
Nrn4,45,21,945
Nrn5,56,82,4592
Nrn6,75,88,6600
Nrn7,63,50,3150
Nrn8,33,61,2013
Nrn9,46,39,1794


### Removing columns is also easy and done on the fly... 

In [35]:
# using the del command will delete a column from the DF
# note that here you have to use the df['resp3'] notation
# the df.resp3 notation will not work.
del df['resp3']

KeyError: 'resp3'

### Instead of deleting outright, you can also "pop" a column out and asign it to another variable

In [36]:
# define a third response column as the product of the first two columns
df['resp3'] = df.resp1 * df.resp2

# then pop it out
resp3 = df.pop('resp3')
display(resp3)

# now df is back down to just 2 columns
print('\n')
display(df)

Nrn0     434
Nrn1    1080
Nrn2     868
Nrn3    4488
Nrn4     945
Nrn5    4592
Nrn6    6600
Nrn7    3150
Nrn8    2013
Nrn9    1794
Name: resp3, dtype: int64

,resp1,resp2
Nrn0,14,31
Nrn1,27,40
Nrn2,62,14
Nrn3,88,51
Nrn4,45,21
Nrn5,56,82
Nrn6,75,88
Nrn7,63,50
Nrn8,33,61
Nrn9,46,39


## More on indexing and selection of specific coordinates in a DF

### Row selection - this is a bit more confusing as there are many methods
* You can use df.loc to select a row by its label name
* You can use df.iloc to select a row by its integer location 
* You can use boolean vectors to select a set of rows that satisfy some condition
* You can slice rows using standard notation e.g. df[1:3] for rows 1-3
* You can also isolate a particular row/column using a combo of column indexing (see above) and standard slicing notation

<div class="alert alert-danger">
Contrary to usual slicing conventions, both the start and the stop indices are included when using the DF.LOC function...see below for demo. This makes sense (sort of) because you're indexing by label name, not by a zero-based integer index. 
</div>

In [37]:
# data from 2nd neuron across both stimulus conditions
df.loc['Nrn1']

resp1    27
resp2    40
Name: Nrn1, dtype: int64

In [38]:
# CAREFUL!
# data from 2nd-6th neuron inclusive across both stimulus conditions
df.loc['Nrn1':'Nrn5']

# again, just need to be careful but this makes sense given that you're indexing based on label name (not 0-based counting)

,resp1,resp2
Nrn1,27,40
Nrn2,62,14
Nrn3,88,51
Nrn4,45,21
Nrn5,56,82


In [39]:
# data from 5th neuron across both stimulus conditions
df.iloc[4]

resp1    45
resp2    21
Name: Nrn4, dtype: int64

In [40]:
# data from 2nd-5th neuron across both stimulus conditions
df[2:5]

,resp1,resp2
Nrn2,62,14
Nrn3,88,51
Nrn4,45,21


In [41]:
# can use the trick for returning only a subset of values from a function that we discussed in
# the randomization/bootstrapping lecture:
# here grab the 2nd entry from the 2nd column
print('2nd column, 2nd entry')
print(df['resp2'][1])

# can also go like this
print('2nd column, 4th entry')
print(df.resp2[3])

2nd column, 2nd entry
40
2nd column, 4th entry
51


## Data alignment - like with the Series object, but accounts for a union of both column and index(row) labels
* Recall from the tutorial on Series objects that if you have mismatched or different length Series, you can still combine them and do operations on them.
* The missing/mismatched elements in the resulting Series was marked with a NaN, and the same will happen with a DF

In [42]:
# make two dfs and then combine them...

# first df will be 3 x 4
r0 = 3
c0 = 4

df0 = pd.DataFrame(np.random.rand(r0,c0), index=['r1', 'r2', 'r3'], columns=['a', 'b', 'c', 'd'])

# second df will be 4 x 5
r1 = 4
c1 = 5

df1 = pd.DataFrame(np.random.rand(r1,c1), index=['r1', 'r2', 'r3','r4'], columns=['a', 'b', 'c', 'd','e'])

# now combine using a unary operation (lets do element by element multiplication)

df_prod = df0 * df1

# note the NaNs in all the places where there is a mismatch between the two original dfs
display(df_prod)

,a,b,c,d,e
r1,0.578232,0.321536,0.112625,0.018429,NaN
r2,0.706624,0.047246,0.121336,0.139730,NaN
r3,0.880843,0.097544,0.058006,0.459186,NaN
r4,NaN,NaN,NaN,NaN,NaN


## Similar to above example, but now make two dfs that are the same size, but only a subset of the labels match
* To demonstrate that combining dfs relies on a union of both row (index) and column labels, have one of each mismatch 
* Write this as if we made a common coding error and mislabeled some of index/column labels

In [43]:
# first df will be 4 x 4
r0 = 4
c0 = 4

df0 = pd.DataFrame(np.random.rand(r0,c0), index=['r1', 'r2', 'rr3','r4'], columns=['a', 'b', 'c', 'd'])

# second df will be 4 x 5
r1 = 4
c1 = 4

df1 = pd.DataFrame(np.random.rand(r1,c1), index=['r1', 'r2', 'r3','r4'], columns=['a', 'b', 'cv', 'd'])

# now combine using a unary operation (lets do element by element multiplication)

df_prod = df0 * df1

# note the NaNs in all the places where there is a mismatch 
# between the index/column labels in two original dfs
display(df_prod)

,a,b,c,cv,d
r1,0.305230,0.440671,NaN,NaN,0.006637
r2,0.641345,0.555083,NaN,NaN,0.279995
r3,NaN,NaN,NaN,NaN,NaN
r4,0.357056,0.206229,NaN,NaN,0.554470
rr3,NaN,NaN,NaN,NaN,NaN


## Just as with the Series objects, we can use most NumPy operations on DFs without any trouble 

In [44]:
# true matrix mulitplication (not element by element)
np.dot(df0, df1)

array([[1.48523886, 1.28763928, 1.17310353, 0.86308584],
       [1.97161741, 1.83031575, 1.54511031, 1.04371387],
       [1.39663958, 1.44353506, 1.10836406, 0.62050821],
       [1.84497689, 1.47258831, 1.36236537, 1.12956949]])

## Get a high-level summary of the data using built-in functionality of DataFrame object
[API reference page](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html)

In [45]:
# first call this using the defaults
df.describe()

,resp1,resp2
count,10.000000,10.000000
mean,50.900000,47.700000
std,22.521348,24.166322
min,14.000000,14.000000
25%,36.000000,33.000000
50%,51.000000,45.000000
75%,62.750000,58.500000
max,88.000000,88.000000


In [46]:
# can specify different values to change behavior...
df.describe(percentiles=np.linspace(0,1,11))

,resp1,resp2
count,10.000000,10.000000
mean,50.900000,47.700000
std,22.521348,24.166322
min,14.000000,14.000000
0%,14.000000,14.000000
10%,25.700000,20.300000
20%,31.800000,29.000000
30.0%,41.400000,36.600000
40%,45.600000,39.600000
50%,51.000000,45.000000


## And also just like the Series object, you can also apply many other standard analyses using the DF object directly

* [Pandas doc for all functions](https://pandas.pydata.org/pandas-docs/stable/api.html#api-dataframe-stats)

In [47]:
df.mean(axis=0)

resp1    50.9
resp2    47.7
dtype: float64

## Given our main DF with the data from the simulated unit recording study, demonstrate some simple file I/O
* Start by saving the data in a csv (comma separated values) file, which is a common format that is useful for small-to-medium sized data and can easily store mixed data types (strs, ints, floats, etc)

[more than you ever wanted to know about the CSV format](https://www.loc.gov/preservation/digital/formats/fdd/fdd000323.shtml)

[API reference page for csv read/write in Pandas](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html)

In [52]:
# Check out API reference page for all the flags, but for the moment
# the we'll save our header in the file so that reading it back in is easier...
# can do this either way, you just need to know how a file was written when 
# go to read it 
df.to_csv('spike_rates.csv',index=True,header=True)

### Have a look at the csv file in a text editor...

In [49]:
# use our current working directory to build a path to the file
print(cwd)
file_name = cwd + '/spike_rates.csv'
print(file_name)

# read back in the .csv file that we made above. 
# by default read_csv will try to infer the column headers but since
# we explicitly wrote them out (header=True on write) we can tell 
# read_csv that row 0 in the file has the header info...
# same for the row labels (index labels)...tell
# read_csv that our row labels are in the first column (0) 
# of the file
df = pd.read_csv(file_name, index_col=0, header=0)
df

C:\Users\Chloe\Documents\GitHub\PSYC-NEU-231
C:\Users\Chloe\Documents\GitHub\PSYC-NEU-231/spike_rates.csv


,resp1,resp2
Nrn0,14,31
Nrn1,27,40
Nrn2,62,14
Nrn3,88,51
Nrn4,45,21
Nrn5,56,82
Nrn6,75,88
Nrn7,63,50
Nrn8,33,61
Nrn9,46,39


## Making cooler DataFrame styles (and more useful...although that should take a backseat to coolness)
[Check here for a bunch of neat style options](https://pandas.pydata.org/pandas-docs/stable/style.html)
* Simple demo - can write custom functions that highlight specific aspects of your data - can be very useful for more clearly highlighting/communicating key points in the data within a notebook  

In [50]:
# highlight the max value in each column in yellow...
def highlight_max_value_in_columns(data_frame):
    ind_max = data_frame == data_frame.max()
    return ['background-color: yellow' if i else '' for i in ind_max]


In [51]:
# apply it here!
df.style.apply(highlight_max_value_in_columns)

,resp1,resp2
Nrn0,14,31
Nrn1,27,40
Nrn2,62,14
Nrn3,88,51
Nrn4,45,21
Nrn5,56,82
Nrn6,75,88
Nrn7,63,50
Nrn8,33,61
Nrn9,46,39
